In [ ]:
# Install required packages
!pip install diffusers transformers xformers git+https://github.com/huggingface/accelerate.git
!pip install opencv-contrib-python controlnet_aux

In [1]:
# Import required libraries
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
from PIL import Image
import numpy as np
import cv2
from google.colab import files
from io import BytesIO


In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [3]:
# Enable GPU acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize ControlNet model with optimizations
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16
)

# Initialize pipeline with optimizations
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16
)

# Configure pipeline for optimal performance
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_model_cpu_offload()

Using device: cuda


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving jeans.jpg to jeans.jpg
Saving nike-hoodie.jpg to nike-hoodie.jpg
Saving shirt.jpg to shirt.jpg
User uploaded file "jeans.jpg" with length 391019 bytes
User uploaded file "nike-hoodie.jpg" with length 255425 bytes
User uploaded file "shirt.jpg" with length 40585 bytes


In [6]:
# Helper function for image processing
def process_and_blend_images(image_paths, size=(512, 512)):
    """
    Process multiple images and blend their Canny edge maps into one control image.
    """
    edge_accumulator = None
    for path in image_paths:
        image = Image.open(path).resize(size).convert("RGB")
        np_image = np.array(image)
        edges = cv2.Canny(np_image, 100, 200)
        edges_rgb = np.stack([edges]*3, axis=-1)  # Convert to RGB-like array

        if edge_accumulator is None:
            edge_accumulator = edges_rgb.astype(np.float32)
        else:
            edge_accumulator += edges_rgb.astype(np.float32)

    # Average edges
    averaged_edges = (edge_accumulator / len(image_paths)).astype(np.uint8)
    control_image = Image.fromarray(averaged_edges).convert("RGB")
    return control_image

def generate_design(prompt, control_images, num_steps=30, guidance_scale=7.5): # num steps reduced from 30 --> 5
    """
    Generate designs using the ControlNet pipeline for multiple control images.

    Args:
        prompt (str): The text prompt to guide generation.
        control_images (List[PIL.Image]): List of processed control images.
        num_steps (int): Number of inference steps.
        guidance_scale (float): Classifier-free guidance scale.

    Returns:
        List[PIL.Image]: Generated images for each control image.
    """
    results = []
    for i, control_image in enumerate(control_images):
        print(f"Generating image {i+1}/{len(control_images)}...")
        result = pipe(
            prompt=prompt,
            image=control_image,
            num_inference_steps=num_steps,
            guidance_scale=guidance_scale
        ).images[0]
        results.append(result)
    return results

In [8]:
    image_paths = [
        "jeans.jpg",
        "nike-hoodie.jpg",
        "shirt.jpg"
    ]

    # Create one blended control image
    control_image = process_and_blend_images(image_paths)
      # Gemini-flash-prompt needs to be shortened to under 80 tokens to work
    prompt = (
        "Create a lined denim jacket using three materials. Dark navy denim (jeans.jpg) forms the durable body."
        "The nke-hoodie.jpg material makes a detachable hood with water resistance."
        "Plaid cotton blend from shirt.jpg lines the jacket, adding warmth."
    )

    # Generate a single final design
    result = generate_design(prompt, [control_image])[0]

    result.save("final_design.jpg")
    result.show()

Generating image 1/1...


  0%|          | 0/30 [00:00<?, ?it/s]